The idea is to build a model to take as input a sequence of characters representing a name, and find the associated country.

Inspired by: https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial

# Imports

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import os
import random

# Data

Reading the data from the files.

In [2]:
categories = []
category_lines = {}

for (dirpath, dirnames, filenames) in os.walk('data/names'):
    for (i, filename) in enumerate([dirpath + os.sep + f for f in filenames]):
        category = os.path.splitext(os.path.basename(filename))[0]
        categories.append(category)
        lines = open(filename, encoding='utf-8').read().lower().strip().split('\n')
        category_lines[category] = lines

In [3]:
def str_to_tensor(name: str) -> torch.Tensor:
    tensor = torch.zeros(len(name), 1, 512)
    for (i, c) in enumerate(name):
        tensor[i] = F.one_hot(torch.Tensor([ord(c)]).long(), num_classes=512)
        
    return tensor

# Neural network module

In [4]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        
        self.hidden_size = hidden_size
        
        self.i2h = nn.Linear(input_size, hidden_size)
        self.h2h = nn.Linear(hidden_size, hidden_size)
        self.h2o = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
        
    def forward(self, input, hidden):
        hidden = F.tanh(self.i2h(input) + self.h2h(hidden))
        output = self.h2o(hidden)
        output = self.softmax(output)
        return output, hidden
        
    def init_hidden(self):
        return torch.zeros(1, self.hidden_size)

In [5]:
n_hidden = 128
rnn = RNN(512, n_hidden, len(categories))

# Training

In [6]:
criterion = nn.NLLLoss()
learning_rate = 0.005
n_iters = 100000

In [7]:
def get_training_example():
    category = random.choice(list(categories))
    line = random.choice(category_lines[category])
    category_tensor = torch.tensor([categories.index(category)], dtype=torch.long)
    line_tensor = str_to_tensor(line)
    return category_tensor, line_tensor

In [10]:
def train(category_tensor, line_tensor):
    hidden = rnn.init_hidden()
    
    rnn.zero_grad()
    
    for i in range (line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)
        
    loss = criterion(output, category_tensor)
    loss.backward()
    
    for p in rnn.parameters():
        p.data.add_(p.grad.data, alpha=-learning_rate)
        
    return output, loss.item()

In [11]:
current_loss = 0
for iter in range(1, n_iters + 1):
    category_tensor, line_tensor = get_training_example()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

# Evaluation of results

In [ ]:
line_tensor = str_to_tensor("satoshi")

hidden = rnn.init_hidden()

for i in range (line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)
        
print(categories[output.argmax().item()])